<div style="text-align: center; background-color: #b1d1ff; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Stage 02 - Preprocessing
</div>

---

## Import library

In [1]:
import numpy as np
import pandas as pd
import os
import sys
import re

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Read data

In [3]:
data = pd.read_csv('../data/raw_data.csv')
data.head()

,job_title,company_name,salary,address,date_posted,industry,company_size,company_nationality,experience_years,position_level,employment_type,contract_type,technologies_used,job_description,company_url,job_url
0,UX/UI Designer (Game),CÔNG TY CỔ PHẦN SUNTEK,Thương lượng,"['Thành phố Thủ Đức, Hồ Chí Minh']",['Đăng 1 giờ trước'],['Giải trí/ Game'],['25-99 Nhân viên'],['Thailand'],['Từ 2 năm'],"['Junior', 'Middle']",['In Office'],['Fulltime'],"['UX/UI Design', 'HTML & CSS', 'UI/UX']",['Trách nhiệm công việc\n1. General task\nHave...,['https://topdev.vn/nha-tuyen-dung/cong-ty-co-...,https://topdev.vn/viec-lam/ux-ui-designer-game...
1,Senior DevOps Engineer,CÔNG TY CỔ PHẦN CÔNG NGHỆ ỨNG DỤNG XANH VIỆT NAM,40.000.000 VND to 50.000.000 VND,"['Quận Ba Đình, Hà Nội']",['Đăng 3 giờ trước'],[],['Nhân viên'],[],['Từ 5 năm'],['Senior'],['In Office'],['Fulltime'],"['DevOps', 'Docker', 'Ansible', 'CI/CD', 'Terr...","['GA là một công ty khởi nghiệp, phát triển và...",['https://topdev.vn/nha-tuyen-dung/cong-ty-co-...,https://topdev.vn/viec-lam/senior-devops-engin...
2,KỸ SƯ PHẦN MỀM (C# Developer) - Urgent,Aram Vina,Lên tới 1.500 VND,"['Quận Tây Hồ, Hà Nội']",['Đăng 5 giờ trước'],"['Thiết Kế, Software, Gia công phần mềm, Triển...",['25-99 Nhân viên'],['South-Korea'],['Không yêu cầu'],"['Fresher', 'Junior', 'Middle']",['In Office'],['Fulltime'],"['C#', 'WPF', 'Software Developer']",['CÔNG TY TNHH ARAM VINA cần tuyển dụng GẤP KỸ...,['https://topdev.vn/nha-tuyen-dung/aram-vina-9...,https://topdev.vn/viec-lam/ky-su-phan-mem-c-de...
3,Chuyên viên Thẩm định mua sắm CNTT,Ban Công nghệ BIDV,Thương lượng,"['Quận Hai Bà Trưng, Hà Nội']",['Đăng 5 giờ trước'],['Ngân Hàng'],['Hơn 1000 Nhân viên'],['Vietnam'],['Từ 3 năm'],['Middle'],['In Office'],['Fulltime'],"['IT Support', 'IT Service']",['Trách nhiệm công việc\nThực hiện công tác th...,['https://topdev.vn/nha-tuyen-dung/ban-cong-ng...,https://topdev.vn/viec-lam/chuyen-vien-tham-di...
4,Chuyên viên Quản lý Chuyển đổi số,Ban Công nghệ BIDV,Thương lượng,"['Quận Hai Bà Trưng, Hà Nội']",['Đăng 5 giờ trước'],['Ngân Hàng'],['Hơn 1000 Nhân viên'],['Vietnam'],['Từ 2 năm'],"['Junior', 'Middle']",['In Office'],['Fulltime'],"['Agile', 'Project Manager', 'Product Owner', ...",['Trách nhiệm công việc\nPhối hợp triển khai t...,['https://topdev.vn/nha-tuyen-dung/ban-cong-ng...,https://topdev.vn/viec-lam/chuyen-vien-quan-ly...


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   job_title            984 non-null    object
 1   company_name         984 non-null    object
 2   salary               983 non-null    object
 3   address              984 non-null    object
 4   date_posted          984 non-null    object
 5   industry             984 non-null    object
 6   company_size         984 non-null    object
 7   company_nationality  984 non-null    object
 8   experience_years     984 non-null    object
 9   position_level       984 non-null    object
 10  employment_type      984 non-null    object
 11  contract_type        984 non-null    object
 12  technologies_used    984 non-null    object
 13  job_description      984 non-null    object
 14  company_url          984 non-null    object
 15  job_url              984 non-null    object
dtypes: objec

## Processing 

### Tỷ lệ giá trị thiếu

In [12]:
def missing_values(data):
    """
    This function takes a DataFrame and returns a DataFrame with the percentage of missing values for each column.
    """
    missing = data.isnull().sum()
    missing_prob = missing / len(data) * 100
    missing = pd.DataFrame({'missing': missing, 'missing_prob': missing_prob})
    missing = missing.sort_values(by='missing_prob', ascending=False)

    return missing

missing = missing_values(data)
missing

,missing,missing_prob
salary,1,0.101626
job_title,0,0.000000
company_name,0,0.000000
address,0,0.000000
date_posted,0,0.000000
industry,0,0.000000
company_size,0,0.000000
company_nationality,0,0.000000
experience_years,0,0.000000
position_level,0,0.000000


### Chuẩn hóa giá trị